In [10]:
print("Hmmm, not sure why this is'nt working")

Hmmm, not sure why this is'nt working


# High-level LSTM PyTorch Example

In [1]:
import os
import sys
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
import torch.nn.init as init
from torch import autograd
from torch.autograd import Variable
from common.params_lstm import *
from common.utils import *

In [2]:
# Not sure if this affects LSTMS?
torch.backends.cudnn.benchmark=True # enables cudnn's auto-tuner

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("PyTorch: ", torch.__version__)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
PyTorch:  0.2.0_1
Numpy:  1.13.1
GPU:  ['Tesla K80', 'Tesla K80']


In [4]:
class SymbolModule(nn.Module):
    def __init__(self):
        super(SymbolModule, self).__init__()
        # http://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html
        self.hidden_dim = NUMHIDDEN
        
        self.embed = nn.Embedding(MAXFEATURES, EMBEDSIZE)
        self.lstm = nn.LSTM(EMBEDSIZE, NUMHIDDEN)
        
        self.hidden2tag = nn.Linear(NUMHIDDEN, 2)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (Variable(torch.zeros(1, 1, self.hidden_dim)),
                Variable(torch.zeros(1, 1, self.hidden_dim)))

    def forward(self, x):
        embeds = self.embed(x)
        lstm_out, self.hidden = self.lstm(
            embeds.view(MAXLEN, 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(MAXLEN, -1))
        return tag_space

In [5]:
def init_model(m):
    # Implementation of momentum:
    # v = \rho * v + g \\
    # p = p - lr * v
    opt = optim.Adam(m.parameters(), lr=LR, betas=(BETA_1, BETA_2), eps=EPS)
    criterion = nn.CrossEntropyLoss()
    return opt, criterion

In [15]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = imdb_for_library(seq_len=MAXLEN, max_features=MAXFEATURES)# CNTK format
# Torch-specific
x_train = x_train.astype(np.int64)
x_test = x_test.astype(np.int64)
y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Trimming to 25000 max-features
Padding to length 150
(25000, 150) (25000, 150) (25000,) (25000,)
int64 int64 int64 int64
CPU times: user 5.84 s, sys: 252 ms, total: 6.09 s
Wall time: 7.16 s


In [7]:
%%time
sym = SymbolModule()
sym.cuda() # CUDA!

CPU times: user 2.42 s, sys: 504 ms, total: 2.93 s
Wall time: 2.94 s


In [8]:
%%time
optimizer, criterion = init_model(sym)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 139 µs


In [19]:
# Sets training = True
sym.train()  
for j in range(EPOCHS):
    for data, target in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        # Get samples
        data = Variable(torch.LongTensor(data).cuda())
        target = Variable(torch.LongTensor(target).cuda())
        # Init
        optimizer.zero_grad()
        # Clear out hidden state of lstm
        sym.hidden = sym.init_hidden()
        # Forwards
        output = sym(data)
        # Loss
        loss = criterion(output, target)
        # Back-prop
        loss.backward()
        optimizer.step()
    # Log
    print(j)

RuntimeError: input.size(-1) must be equal to input_size. Expected 150, got 9600

In [ ]:
%%time
# Test model
# Sets training = False
sym.eval()
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, target in yield_mb(x_test, y_test, BATCHSIZE):
    # Get samples
    data = Variable(torch.FloatTensor(data).cuda())
    # Forwards
    output = sym(data)
    pred = output.data.max(1)[1].cpu().numpy().squeeze()
    # Collect results
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = pred
    c += 1

In [ ]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))